In [1]:
from datetime import datetime
import os
import sys
import time

import numpy as np
import pandas as pd

from sherlock import helpers
from sherlock.deploy.model import SherlockModel
from sherlock.functional import extract_features_to_csv
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import (
    extract_features,
    convert_string_lists_to_lists,
    prepare_feature_extraction,
    load_parquet_values,
)
from sherlock.features.word_embeddings import initialise_word_embeddings

import sklearn.metrics as skl
import altair as alt
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [2]:
model = SherlockModel();
model.initialize_model_from_json(with_weights=True, model_id="sherlock");

W0621 10:19:40.759631 139953414031168 deprecation.py:506] From /home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0621 10:19:40.761058 139953414031168 deprecation.py:506] From /home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0621 10:19:40.764913 139953414031168 deprecation.py:506] From /home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Varia

In [3]:
# path_features = '../data/data/processed/test_20220502-202626.csv'
path_features = '../data/data/processed/validation_20220307-132110.csv'
path_true_types = '../data/data/raw/val_labels.parquet'
# path_temp_shuffled = '../data/data/processed/temp_features_shuffled.csv'
LOOP_COUNT = 4 # If the notebook crashed it could be due to high ram usage, if so, try to lower the loop count
# indices of columns of each feature set, uncumment either row wise permutation or column wise depending on the one you want to perform.
feature_cols_dict = {
    # row wise permutation
    'character_distributions': [range(0, 960)],
    'word_embeddings': [range(960, 1161)],
    'global_statistics': [range(1161, 1188)],
    'paragraph_vectors': [range(1188, 1588)],
    # column wise permutation
#     'character_distributions': list(range(0, 960)),
#     'word_embeddings': list(range(960, 1161)),
#     'global_statistics': list(range(1161, 1188)),
#     'paragraph_vectors': list(range(1188, 1588)),
}
# the last feature of the word embeddings is a binary value that indicates 
# if the word embeddings were successfully extracted for that specific column. 1160


In [4]:
df_features_original = pd.read_csv(path_features)
true_type_labels = pd.read_parquet(path_true_types).values.flatten()

In [5]:
shuffled_features = {}
shuffled_f1_scores_dict = {}

for k, v in feature_cols_dict.items():
    shuffled_features[k] = []
    shuffled_f1_scores_dict[k] = []

for k, v in feature_cols_dict.items():
    for i in range(LOOP_COUNT):
        print(f'shuffling index {i}')
        df_features_shuffled = df_features_original.copy()
        for col_idx in feature_cols_dict[k]:
            df_features_shuffled.iloc[:, col_idx] = df_features_shuffled.iloc[:, col_idx].sample(frac=1, replace=True).reset_index(drop=True)
            print(f'shuffled index {i}')
        predictions = model.predict(df_features_shuffled, "sherlock")
        shuffled_f1_scores_dict[k].append(skl.classification_report(true_type_labels, predictions,output_dict=True)['macro avg']['f1-score'])
#         shuffled_features[k].append(df_features_shuffled)
        df_features_shuffled = None


shuffling index 0
shuffled index 0
shuffling index 1
shuffled index 1
shuffling index 2
shuffled index 2
shuffling index 3
shuffled index 3
shuffling index 0
shuffled index 0
shuffling index 1
shuffled index 1
shuffling index 2
shuffled index 2
shuffling index 3
shuffled index 3
shuffling index 0
shuffled index 0
shuffling index 1
shuffled index 1
shuffling index 2
shuffled index 2
shuffling index 3
shuffled index 3
shuffling index 0
shuffled index 0
shuffling index 1
shuffled index 1
shuffling index 2
shuffled index 2
shuffling index 3
shuffled index 3


In [6]:
# for df_features_shuffled in list_df_features_shuffled:
#     print(df_features_shuffled.head())

In [7]:
# for col_idx in feature_cols_dict[feature_to_shuffle]:
#     #find count of different values for each column between df_features_original and df_features_shuffled
#     df_features_original_col = df_features_original.iloc[:, col_idx]
#     df_features_shuffled_col = list_df_features_shuffled[0].iloc[:, col_idx]
#     # find the amount of non equal values between the colmuns
#     comparison_column = np.where(df_features_original_col == df_features_shuffled_col, True, False)
#     #print amount of true values in comparison_column
#     print(np.sum(comparison_column))


In [8]:
# for col_idx in feature_cols_dict[feature_to_shuffle]:
#   print(df_features_original.columns[col_idx])

In [9]:
# TODO REMOVE
# original: f1 ~ 0.874
# predicted_labels_original = model.predict(df_features_original, "sherlock")
# print(skl.classification_report(true_type_labels, predicted_labels_original,output_dict=True)['weighted avg'])

In [10]:
# !!Do it also on the benchmark an compare results!!
# character_distributions: f1 ~ row wise 0.822 || col wise 0.835
# word_embeddings: f1 ~ row wise 0.820 || col wise 0.0015 -> Glove 50 dim rep. -> mean(avg. word vector each dimension gets permuted indep., same for other aggregates), mode, ... -> we are permuting these vectors in col wise
# global_statistics: f1 ~ row wise 0.854 || col wise 0.856
# paragraph_vectors: f1 ~ row wise 0.225 || col wise 0.369 -> col wise mixes topic completely, could indicate that other features get more importance when predicting, compared to a wrongly given topic //+- true, given a certain window of values, predict the rest of window. paragraph vec subnetwork should already be permutation invariant. Could be an explanation why f1 scores are close 
# for df_features_shuffled in list_df_features_shuffled:
#     predicted_labels_shuffled = model.predict(df_features_shuffled, "sherlock")
#     print(skl.classification_report(true_type_labels, predicted_labels_shuffled,output_dict=True)['weighted avg'])



In [11]:
# predicted_labels_dict = {}
# shuffled_f1_scores_dict = {}

# for k, v in feature_cols_dict.items():
#     predicted_labels_dict[k] = []
#     shuffled_f1_scores_dict[k] = []

# for k, v in shuffled_features.items():
#     for shuffled_feature_set in v:
#         predictions = model.predict(shuffled_feature_set, "sherlock")
#         predicted_labels_dict[k].append(predictions)
#         shuffled_f1_scores_dict[k].append(skl.classification_report(true_type_labels, predictions,output_dict=True)['weighted avg']['f1-score'])

In [12]:
print(shuffled_f1_scores_dict)

{'character_distributions': [0.8018761071510504, 0.8028221793214116, 0.8034173031895644, 0.8022839359158281], 'word_embeddings': [0.8068624624563837, 0.8050008061751265, 0.8069130863470321, 0.8063149091191081], 'global_statistics': [0.837698111894723, 0.8379098564421205, 0.8363609668287273, 0.8368823938817931], 'paragraph_vectors': [0.19500709969517993, 0.19634533726544803, 0.19414038135673614, 0.1940058051397236]}


In [13]:
df_dict = {'feature': list(shuffled_f1_scores_dict.keys()), 'avg f1-score': list(map(lambda x: sum(x)/len(x), shuffled_f1_scores_dict.values()))}
col_amount_freq_df = pd.DataFrame(df_dict)
alt.Chart(col_amount_freq_df.reset_index()).mark_bar(size=40).encode(
    x = alt.X('feature',
              type='ordinal',
              title = 'Shuffled feature',
#               sort=alt.EncodingSortField(
#                 field="values",  
#                 order="descending"),
              ),
    y = alt.Y('avg f1-score', title='Average F1-score'),
).configure_axis(
    labelFontSize=18,
    titleFontSize=18
).configure_axisX(
    labelAngle=0
).properties(width=800,height=200)

alt.Chart(...)